# Sentence Completion
This notebook is used for a final project as a part of the CS 4980 Natural Language Processing course at the Milwaukee School of Engineering during the spring term of 2023. This notebook was created by [Grant Fass](grantfass@gmail.com) and [Nicholas Kaja](kajan@msoe.edu). The following notebook will explore the problem of natural language sentence completion. This notebook can also be reached directly through the following GitHub repository: [https://github.com/GrantFass/NLP/tree/main/Sentence%20Completion](https://github.com/GrantFass/NLP/tree/main/Sentence%20Completion)

Natural language sentence completion involves determining what word best fits in a blank present in a sentence. This type of question is typically found on the Scholastic Aptitude Test (SAT). It is useful because it can measure the performance of a language model (LM) on questions that educational experts deem important. LMs that perform well on this type of problem will likely perform better on broader tasks.

The primary method for forming a sentence completion model is to compute the probability of each possible sentence tehn choose the most probable option. This probability computation can be done using n-grams, Latent Semantic Analysis (LSA), and Syntactic Dependency Trees. Some research has also been done into combining these with methods of preserving long-range dependencies in the sentences. Of these options, N-grams is one of the easiest starting points due to its ease of implementation and understanding. N-grams also allow for sufficient variations over the base model such as different N-gram algorithms, different values of N, and different methods of tokenization.

We will be constructing our models from a dataset of Khan Academy lecture transcripts. This dataset was scraped using BeautifulSoup during January of 2023 by Nicholas Kaja as a part of a senior design project. The performance of our model will be evaluated on a [SAT Question Dataset](https://github.com/ctr4si/sentence-completion/tree/master/data/completion). After evaluation, we plan to take our best performing model and apply it to sentence generation. This will allow us to get a better feel for how well it performs. Once our model is working, we also plan to implement some additional features such as Named Entity Recognition (NER). If there is enough time we also plan to investigate using the [OpenAI tokenizer](https://platform.openai.com/tokenizer) instead of the [NLTK Word Tokenizer](https://www.nltk.org/api/nltk.tokenize.html) or the [SpaCy Tokenizer](https://spacy.io/api/tokenizer). The OpenAI tokenizer has a [python package found on github](https://github.com/openai/tiktoken).

For more information please see the Data Collection And Processing document as well as the Project Background document. Both of these documents can be found in the repository under the Sentence Completion directory.

## Pip Installations
The below magic command should install all of the required python packages to run this project. If it does not run successfully please try to run the command through an admin shell instead.

In [1]:
%pip install -r requirements.txt --user

Note: you may need to restart the kernel to use updated packages.


## Imports
This cell contains the imports and some setup functions. This is mostly generalized which means there are extra imports as of now.

In [2]:
from pathlib import Path
from collections import Counter, defaultdict
import pandas as pd
import random
from tqdm.notebook import tqdm
import nltk
from nltk.util import ngrams
from nltk.util import everygrams
import nltk.data
import re
import contractions
from bs4 import BeautifulSoup
import unidecode
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
import math
import numpy as np
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
])

lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()
encoder = LabelEncoder()
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
tqdm.pandas()
n = 3

[nltk_data] Downloading package names to /home/fassg/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /home/fassg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /home/fassg/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/fassg/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/fassg/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fassg/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/fassg/nltk_data...
[nltk_data]   Package vader

## Data Loading  
This section is used to load in all of the training data for the vocabulary. This comes from the 5 csv files of Khan Academy lectures. These files are all combined into a single dataframe for easier use and analysis. In the future we may need to include other sources of data such as ted talks or even ebooks in order to widen our vocabulary.

We also load in the SAT dataset here. This is a short dataset that we are primarily using as our testing set. The goal here is to guess which response option, out of five, is the correct response for a fill-in-the-blank SAT question. 

In [3]:
computing_df = pd.read_csv(Path("Datasets/KhanAcademy/Computing.csv"))
computing_df = computing_df.dropna()

economics_df = pd.read_csv(Path("Datasets/KhanAcademy/Economics.csv"))
economics_df = economics_df.dropna()

humanities_df = pd.read_csv(Path("Datasets/KhanAcademy/Humanities.csv"))
humanities_df = humanities_df.dropna()

math_df = pd.read_csv(Path("Datasets/KhanAcademy/Math.csv"))
math_df = math_df.dropna()

science_df = pd.read_csv(Path("Datasets/KhanAcademy/Science.csv"))
science_df = science_df.dropna()

khan_dfs = [computing_df, economics_df, humanities_df, math_df, science_df]
khan = pd.concat(khan_dfs, axis=0)
khan.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8261 entries, 0 to 2789
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       8261 non-null   object
 1   unit         8261 non-null   object
 2   lesson       8261 non-null   object
 3   video_title  8261 non-null   object
 4   about        8261 non-null   object
 5   transcript   8261 non-null   object
dtypes: object(6)
memory usage: 451.8+ KB


In [4]:
sat = pd.read_csv(Path("Datasets/SAT_Question_Dataset.csv"))
sat.info(verbose=True, show_counts=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets\\SAT_Question_Dataset.csv'

## Cleaning
This section is dedicated to cleaning up some of our vocabulary dataset before it is used to create the vocabulary. We have decided, for simplicity, to remove the first sentence of every transcript in the dataset. This will allow us to easily remove all of the tagging information such as "[Instructor]" and "- Speaker 1:". Most of the first sentences are greetings and introductions. As such they are not necessarily as important to answering the SAT question dataset. We have also elected to expand contractions, remove html tags, remove quotes, and replace accented characters.

The same general cleaning method is run over our testing data as well. The first sentence removal is not performed on the testing data.

[Note] Before this section, as a part of the cleaning, we will want to perform SpaCy NER. Use SpaCy NER to identify the multi-word-expressions then use the [nltk.tokenize.mwe module](https://www.nltk.org/api/nltk.tokenize.mwe.html) to combine the tokens - Grant.

In [ ]:
def clean_base(x: str):
        # remove any html tags
        x = BeautifulSoup(x, "html.parser").get_text(separator=" ")
        # # set all to lower
        # x = x.lower()
        # clean up the contractions
        x = contractions.fix(x)
        # remove accended characters
        x = unidecode.unidecode(x)
        # # remove stopwords: https://stackoverflow.com/questions/19560498/faster-way-to-remove-stop-words-in-python
        # x = ' '.join([word for word in x.split() if word not in cachedStopWords]) # slower to use word tokenize
        # # fix punctuation spacing
        # x = re.sub(r'(?<=[\.\,\?])(?=[^\s])', r' ', x)
        # # strip punctuation
        # x = re.sub(r'[\.\,\?\\\/\<\>\;\:\[\]\{\}]', r'', x)
        # strip quotes
        x = x.replace('\'', '').replace('\"', '')
        # remove some actions
        remove_list = ['(Laughter)', '(laughter)', '(Music)', '(music)', '(Music ends)', '(Audience cheers)', '(Applause)', '(Applause ends)', '(Applause continues)', '(Bells)', '(Trumpet)', '(Clears throat)']
        x = ' '.join([word for word in x.split() if word not in remove_list])
        # remove extraneous items
        x = x.replace(' -- ', '').replace(' .. ', ' ').replace(' ... ', ' ')
        # remove extra whitespace
        x = ' '.join(x.strip().split())
        # # may want to add lematization
        # x = ' '.join([lemmatizer.lemmatize(word) for word in x.split()])
        # remove some of the extra bracket tags
        x = re.sub(r"\s{2,}", " ", re.sub(r"[\(\[\{][^\)\]\}]*[\)\]\}]", "", x))
        return x

def remove_first_sentence(doc):
    """
    This removes the first sentence of a document. We use this to remove all narrator / speaker tags, and
    to remove unnecessary introductory sentences that most transcripts have
    """
    return ' '.join(nltk.sent_tokenize(doc)[1:])

transcripts = khan['transcript']
transcripts = transcripts.progress_apply(remove_first_sentence)
transcripts = transcripts.progress_apply(clean_base)
khan['clean_transcript'] = transcripts
khan.head()


In [ ]:
for column_name in ['question', 'a)', 'b)', 'c)', 'd)', 'e)']:
    sat[column_name] = sat[column_name].progress_apply(clean_base)
sat.head()

## N-Grams
This section is used for the tokenization of our text into N-Grams. Various tokenization approaches may yield different results. As such, it is important to test different approaches here. We begin by dividing each of the transcripts into its constituent sentences. These sentences are then each tokenized using the word tokenizer provided by nltk. A start and end of sentence tag is then added to each list of tokens. Next, these tokens are used to generate all possible n-grams below a certain size threshold. Lastly, we create a frequency distribution dictionary for the n-grams.

In [ ]:
# create a list of all of the sentences with each sentence tokenized.
def tokenize_transcript(transcript):
    token_document = [word_tokenize(t) for t in sent_detector.tokenize(transcript)]
    sents = []
    start_of_sentence = "<sent>"
    end_of_sentence = "<\\sent>"
    for sent in token_document:
        sent.insert(0, start_of_sentence)
        sent.append(end_of_sentence)
        sents.append(sent)
    return sents

sents = []
for transcript in tqdm(khan['clean_transcript']):
    sents += tokenize_transcript(transcript)

print(len(sents))

In [ ]:
n = 4
print("Computing all N-grams where N is %d or less" % n)
grams = []
for sent in tqdm(sents):
    grams += list(everygrams(sent, min_len=2, max_len=n))
print("Computing frequency distribution")
freq_dist = nltk.FreqDist(grams)
freq_dist

In [ ]:
# n = 3
# print("Computing all N-grams where N is %d or less" % n)
# grams = []
# for sent in tqdm(sents):
#     for i in range(0, 3):
#         # TODO: change to everygrams.
#         grams += list(ngrams(sent, i + 1))
    
# print("Computing frequency distribution")
# freq_dist = nltk.FreqDist(grams)

In [ ]:
freq_dist

In [ ]:
freq_dist[(',',)]

In [ ]:
freq_dist[('among',)]

## Store Model
The next step is to store the freq_dist model for use in future prediction methods.

In [ ]:
def to_pickle(filename, model):
    with open(filename, 'wb') as fp:
        pickle.dump(model, fp)
        print("Saved %s to file %s" % ('freq_dist', filename))
        
to_pickle(filename='freq_dist.pkl', model=freq_dist)

## Load Model
load the stored model back in as practice.

In [ ]:
def from_pickle(filename):
    model = []
    with open(filename, 'rb') as fp:
        model = pickle.load(fp)
    return model

freq_dist = from_pickle(filename='freq_dist.pkl')
freq_dist

## SAT Dataset
This section focuses on the processing of individual SAT questions as well as defining a prediction method for the dataset. Below is a sample approach for the prediction method.

### SAT Question Prediction Approach
Note that this approach is slightly different from what was implemented. One of the main differences is that our tokenization currently includes punctuation as its own token whereas the below example removes punctuation.

"""  
Question:  
Responding to criticism that the script was rambling and _____, the new screenwriter revised the dialogue for greater succinctness and _____.

Possible Solutions:  
[('engaging', 'simplicity'), ('subjective', 'abiguity'), ('muddled', 'clarity'), ('terse', 'emptiness'), ('difficult', 'abstraction')]  
"""

N-gram size of 4 max
1. Change the blanks out to something we can use to mask the individual ones later. For example, [BLANK1] and [BLANK2].
    - esponding to criticism that the script was rambling and [BLANK1], the new screenwriter revised the dialogue for greater succinctness and [BLANK2].<\sent>
2. Pull out the possible sliding windows centered around each blank.
    - blank 1
        - n=4
            - ('was', 'rambling', 'and', [BLANK1])
            - ('rambling', 'and', [BLANK1], 'the')
            - ('and', [BLANK1], 'the', 'new')
            - ([BLANK1], 'the', 'new', 'screenwriter')
        - n=3
            - ('rambling', 'and', [BLANK1])
            - ('and', [BLANK1], 'the')
            - ([BLANK1], 'the', 'new')
        - n=2
            - ('and', [BLANK1])
            - ([BLANK1], 'the')
        - n=1
            - ([BLANK1])
    - blank 2
        - n=4
            - ('greater', 'succinctness', 'and', [BLANK2])
            - ('succinctness', 'and', [BLANK2], <\sent>)
        - n=3
            - ('succinctness', 'and', [BLANK2])
            - ('and', [BLANK2], <\sent>)
        - n=2
            - ('and', [BLANK2])
            - ([BLANK2], <\sent>)
        - n=1
            - ([BLANK2])
3. For each of the possible solutions:
    1. For each of the windows:
        1. If the first or second word in the possible solution tuple is not in the vocab then discard it as a possible solution.
        2. Replace [BLANK1] with the first item in the possible solution tuple
        3. Replace [BLANK2] with the second item in the possible solution tuple
        4. Lookup each of the window n-grams against the stored n-grams and determine how many times it occured.
        5. What to do if the count for a n-gram is 0 for one word but not for another word?
        6. Calculate the probability of each window n-gram
        7. sum the logs of the probabilities of each window n-gram. 
    2. Determine which of the windows had the highest log likelihood and return the associated word tuple.

In [ ]:
def tokenize_question(question):
    if not isinstance(question, str) or question is None:
        return []
    sent = word_tokenize(question)
    start_of_sentence = "<sent>"
    end_of_sentence = "<\\sent>"
    sent.insert(0, start_of_sentence)
    sent.append(end_of_sentence)
    return sent

def get_mask_indices(row, mask='_____'):
    """Returns a list containing the integer indices of where the mask occurs for a given row of the SAT dataset.

    Args:
        row : a given row of the SAT dataset
        mask (str, optional): the mask to look for the indicies of occurance of. Defaults to '_____'.

    Returns:
        list: contains integer indices of where the mask occurs for a given row of the SAT dataset
    """
    mask_indices = []
    previous_idx = 0
    for blank_id in range(row['blanks']):
        idx = tokenize_question(row['question']).index(mask, previous_idx)
        mask_indices.append(idx)
        previous_idx = idx + 1
    return mask_indices

def extract_possible_solutions(row, mask_indices):
    """computes the possible solutions that can fill in the blanks for a given question in a row of the SAT dataset.
    The possible solutions come from the a-e columns of the row while the question itself comes from the question column.

    Args:
        row (_type_): a given row of the SAT dataset.
        mask_indices (list): list containing integer indices of where the mask occurs for a given row of the SAT dataset.

    Returns:
        list: a list of tuples. Each tuple contains the possible solutions in their respective order to the question.
    """
    possible_solutions = []
    for i in ['a)', 'b)', 'c)', 'd)', 'e)']:
        if i in row:
            # print(test[i])
            tokens = tokenize_question(row[i])
            if tokens:
                possible_solution = []
                for mask_idx in mask_indices:
                    possible_solution.append(tokens[mask_idx])
                possible_solutions.append(tuple(possible_solution))
    return possible_solutions

def get_ngrams_with_mask(mask_indices, tokenized_question, n, mask='_____'):
    """This method is used to compute the sliding window for the original n-gram size as well
    as each of the subsequently smaller sizes of n. This method requires the variable 'n' to
    be defined globally for the maximum n-gram size to look for.

    Args:
        mask_indices (list): list containing integer indices of where the mask occurs for a given row of the SAT dataset.
        tokenized_question (list): list comprised of the individual tokens that were parsed from the input question
        n (int): the maximum n-gram size to use.
        mask (str, optional): the mask to look for the indicies of occurance of. Defaults to '_____'.
        

    Returns:
        list: A list containing tuples of each n-gram is returned. If more than one 
    """
    ranges = []
    count = 0
    for mask_idx in mask_indices:
        remask = mask + str(count)
        tokenized_question[mask_idx] = remask
        count += 1
        # print("\nmask_idx = %d" % mask_idx)
        for i in range(n, 0, -1):  # for each successively smaller n-gram size
            # print("n = %d" % i)
            upper_bound = mask_idx + 1
            lower_bound = mask_idx - i + 1
            if lower_bound >= 0 and upper_bound < len(tokenized_question):
                ranges.append((lower_bound, upper_bound))
                # print("range(%d, %d)" % (lower_bound, upper_bound))
            for j in range(1, i):  # already processed first n-gram for size i. Now process the rest where j is the offset.
                upper_bound += 1
                lower_bound += 1
                if lower_bound >= 0 and upper_bound < len(tokenized_question):
                    ranges.append((lower_bound, upper_bound))
                    # print("range(%d, %d)" % (lower_bound, upper_bound))
    n_grams = []
    for indices in ranges:
        n_gram = []
        for i in range(indices[0], indices[1]):
            n_gram.append(tokenized_question[i])
        n_grams.append(n_gram)
        
    return n_grams
    
    
print("compute the windows")
test = sat.iloc[1]
mask = "_____"
mask_indices = get_mask_indices(test)
tokenized_question = tokenize_question(test['question'])
possible_solutions = extract_possible_solutions(test, mask_indices)
windows = get_ngrams_with_mask(mask_indices, tokenized_question, n=3)
print("mask indicies = %s" % str(mask_indices))
print(possible_solutions)
print(windows)
print(tokenized_question)

In [ ]:
def find_best_answer(possible_solutions, windows, freq_dist, mask='_____'):
    """method used to determine which one of the possible solutions is the most likely answer.

    Args:
        possible_solutions (list): list of tuples where each tuple is one of the possible solutions to fill in the blanks in the sentence.
        windows (list): the windows of n-grams. These windows are centered around the locations of the blanks in the sentence.
        freq_dist (dict): a dictionary where the key is a n-gram as a tuple and the value is the frequency count of the n-gram tuple.
        mask (str, optional): the mask to look for the indicies of occurance of. Defaults to '_____'.

    Returns:
        (tuple): the tuple containing the words from the best possible solution predicted.
    """
    log_likelihood_given_solution = []
    for possible_solution in possible_solutions:  # compute the log likelihood of each possible solution.
        # print("\nGenerating Solutions For: %s" % (possible_solution,))
        log_likelihood = 0
        for window in windows:  # go through each of the generated windows and find its probability.
            for i in range(len(possible_solution)):  # replace the masks with each of the possible solution words
                window = list(map(lambda x: x.replace(mask + str(i), possible_solution[i]), window))
            # print(window)
            ngram_size = len(window)
            raw_ngram_count = freq_dist[tuple(window)]
            if raw_ngram_count > 0:  # the ngram occurs in our corpus
                ngram_count_of_prior = 1  # default to 1 for unigrams
                if ngram_size > 1:  # not a unigram so we need to find the count of the 'given' ngram
                    prior = window[0: len(window)-1]
                    ngram_count_of_prior = freq_dist[tuple(prior)]
                vocab = 0
                raw_probability = raw_ngram_count / (ngram_count_of_prior + vocab)
                log_probability = math.log10(raw_probability)
                log_likelihood += log_probability
        # print("Log Likelihood = %.2f" % log_likelihood)
        log_likelihood_given_solution.append((possible_solution, log_likelihood))
    log_likelihood_given_solution.sort(key = lambda x: x[1], reverse = True)
    answer = log_likelihood_given_solution[0][0]
    # print("Best Answer: %s" % (answer, ))
    # print("sorted likelihoods per possible solution:\n%s" % str(log_likelihood_given_solution))
    return answer

best_answer = find_best_answer(possible_solutions, windows, freq_dist)
print("Best Answer: %s" % (best_answer, ))

In [ ]:
def determine_prediction_based_on_best_answer(best_answer, row, mask_indices):
    """method to determine the column name of the best answer

    Args:
        best_answer (tuple): the tuple containing the words from the best possible solution predicted.
        row : a row in the pandas data frame for the SAT dataset.
        mask_indices (list): list containing the indices of where the mask (blanks) occur.

    Returns:
        str: the column name corresponding to the best answer.
    """
    for col in ['a)', 'b)', 'c)', 'd)', 'e)']:
        if col in row:
            tokens = tokenize_question(row[col])
            result = True
            count = 0
            for mask_idx in mask_indices:
                if tokens[mask_idx] != best_answer[count]:
                    result = False
                count += 1
            if result:
                return col.replace(")", "")
        
prediction = determine_prediction_based_on_best_answer(best_answer, test, mask_indices)
print("The predicted answer is in column %s" % prediction)

In [ ]:
def predict(row):
    # parse the incoming question
    mask = "_____"
    mask_indices = get_mask_indices(row, mask=mask)
    tokenized_question = tokenize_question(row['question'])
    possible_solutions = extract_possible_solutions(row, mask_indices)
    windows = get_ngrams_with_mask(mask_indices, tokenized_question, n=3, mask=mask)
    # Look against the model. Changing the freq_dist used should allow us to try different models.
    # TODO: what if we try to use an ensemble of models with different maximum n-gram sizes?
    best_answer = find_best_answer(possible_solutions, windows, freq_dist, mask=mask)  # note that freq_dist is a global here.
    prediction = determine_prediction_based_on_best_answer(best_answer, row, mask_indices)
    return prediction

def score(y_true, y_pred):
    print("%-15s: %.3f" % ("[ACCURACY]", accuracy_score(y_true, y_pred)))
    print("%-15s: %.3f" % ("[PRECISION]", precision_score(y_true, y_pred, average='weighted')))
    print("%-15s: %.3f" % ("[RECALL]", recall_score(y_true, y_pred, average='weighted')))
    print("%-15s: %.3f" % ("[F1-SCORE]", f1_score(y_true, y_pred, average='weighted')))    

# test the prediction method.
prediction = [predict(test)]
print("Prediction: %s" % prediction)
true_value = [test['ans']]
print("True Value: %s" % true_value)
score(true_value, prediction)

## Predictions
This section is used to actually compute the predictions for our test data and our given input frequency distribution. The results of the prediction are then evaluated.

In [ ]:
# skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=False)
# for i, (train_index, test_index) in enumerate(skf.split(X, y)):
predictions = sat.progress_apply(lambda row: predict(row), axis=1)
predictions
true_values = sat['ans']
score(true_values, predictions)

In [ ]:
def record_model_metrics_as_json(y_true, y_pred):
    metrics = {}
    metrics['accuracy_score'] = accuracy_score(y_true, y_pred)
    metrics['precision_score'] = precision_score(y_true, y_pred, average='weighted')
    metrics['recall_score'] = recall_score(y_true, y_pred, average='weighted')
    metrics['f1_score'] = f1_score(y_true, y_pred, average='weighted')
    return metrics

In [ ]:
sat_metrics = record_model_metrics_as_json(true_values, predictions)

In [ ]:
print(classification_report(true_values, predictions))

In [ ]:
conf = confusion_matrix(true_values, predictions)
fig, ax = plt.subplots(figsize=(8,6), dpi=100)
display = ConfusionMatrixDisplay(conf, display_labels=['a', 'b', 'c', 'd', 'e'])
ax.set(title='Confusion Matrix for SAT sentence completion')
display.plot(ax=ax)

In [ ]:
print(Counter(sat['ans']))
print(Counter(predictions))

In [ ]:
sns.countplot(x=predictions)
plt.show()

## Evaluation of Results
Our model does not perform that well currently. We hypothesize that this is due to both the complexity of the words in the SAT question dataset as well as the size of our input dataset. Our input dataset is only comprised of around 600,000 sentences. Additionally, our input dataset comes from spoken lecture transcripts. While these transcripts should be on fairly complex topics, they still contain mostly spoken language. The SAT question dataset prompts are more complex than the language spoken in our input dataset. This leads to a mismatch between our corpus and training data where most of our n-grams either have low counts or do not exist in our corpus. This could be remedied by expanding the size of our input dataset and thusly our corpus. The research paper that our test dataset came from compiled their corpus on over 1.1 billion words in newspaper articles. Even with this large increase in corpus size they still only achieved a prediction accuracy of just over 50% on the testing dataset. This implies that the test dataset is fairly difficult to get high accuracy scores on.

Note that we did not perform cross validation on our system. The primary reason for this is due to cross validation not making sense in our case. We have created a corpus and are trying to compute the accuracy of our corpus using a test set. As we are not directly training a model, it does not make sense to perform cross validation only over our test set.

## Additionl Dataset Note
[Note] One additional area of improvement that we should also test is improving the size of our test set. I found another dataset that will work for this from the paper [SC-Ques: A Sentence Completion Question Dataset for English as a Second Language Learners](https://arxiv.org/abs/2206.12036). They provided a [link to their code and their data](https://github.com/ai4ed/SC-Ques) for research purposes. This, in turn, gave a link to a [dropbox containing their data](https://www.dropbox.com/s/lzznin2hxt6rmft/SC-Ques.tar.gz?dl=0). The data that we would be looking to use to expand our training set would be found in test.jsons and train.jsons. These two files have been preprocessed in the SC-Ques-Preprocessing.ipynb notebook and saved to the individual processed_data_idx.csv files. They were stored as multiple files due to space limitations with GitHub repositories.

The below section is used to read in and join each of the processed_data_idx.csv files. There should be no need to run the SC-Ques-Preprocessing.ipynb script and is only included for posterity.

In [ ]:
directory_path = Path("Datasets/SC-Ques")
sc_ques = pd.DataFrame()
for data_path in directory_path.glob("**/Processed_data_*.csv"):
    print(data_path)
    data = pd.read_csv(data_path)
    sc_ques = pd.concat([sc_ques, data])
sc_ques = sc_ques.drop(columns=['Unnamed: 0'])
sc_ques

In [ ]:
test = sc_ques.iloc[89546]
print(test['question'])
print(test['a)'])
print(test['b)'])
print(test['c)'])
print(test['d)'])


In [ ]:
prediction = [predict(test)]
print("Prediction: %s" % prediction)
true_value = [test['ans']]
print("True Value: %s" % true_value)
score(true_value, prediction)

In [ ]:
predictions = sc_ques.progress_apply(lambda row: predict(row), axis=1)
predictions
true_values = sc_ques['ans']
score(true_values, predictions)

In [ ]:
sc_ques_metrics = record_model_metrics_as_json(true_values, predictions)
sc_ques_metrics

In [ ]:
print(classification_report(true_values, predictions))

In [ ]:
conf = confusion_matrix(true_values, predictions)
fig, ax = plt.subplots(figsize=(8,6), dpi=100)
display = ConfusionMatrixDisplay(conf, display_labels=['a', 'b', 'c', 'd'])
ax.set(title='Confusion Matrix for SAT sentence completion')
display.plot(ax=ax)

## Store the most recent run of training metrics as a pickle file

In [ ]:
metrics = {"sat": sat_metrics, "sc_ques": sc_ques_metrics}
to_pickle(filename="metrics.pkl", model=metrics)

## TODO:
- write up the readme with dependencies
- write up a how to run
- write up how to use.
- pickle the model that we need. This notebook is only for training the model basically
- Need to write up a python script that allows us to load the model (from the git repo so nobody needs to download it) and will suppport sentence completion as well as sentence writing.
- do not want to need to download the csv's either.
- Needs to be by Wednesday.
- Play with removing punctuation during cleaning.
- number to word conversion (look at "ozone contains _____ oxygen atoms")

## Final Presentation
- aimed towards our peers
- need functioning copy of final project
- what it does and how we did it
- how we chose data
- what we did to the data
- how we trained the model
- give or take 10 min long including demonstration
- 12 groups.
- can switch to giving presentations during the last week of class.

In [ ]:
humanities_df = pd.read_csv(Path("Datasets/KhanAcademy/Humanities.csv"))
sample_lecture = humanities_df.iloc[0]['transcript']
sample_lecture = remove_first_sentence(sample_lecture)
sample_lecture = clean_base(sample_lecture)
# sample_lecture

In [ ]:
sample_lecture_sentences = [word_tokenize(t) for t in sent_detector.tokenize(transcript)]

Look at a sample process for a single sentence.

In [ ]:
from nltk.util import pad_sequence
n = 4
# this is the long way of doing things. there is a convenience method available.
list(pad_sequence(sample_lecture_sentences[0],
                  pad_left=True,
                  left_pad_symbol="<s>",
                  pad_right=True,
                  right_pad_symbol="</s>",
                  n=2)) # n denotes the padding for n-grams. probably do not want more than 2.

In [ ]:
from nltk.lm.preprocessing import pad_both_ends
list(pad_both_ends(sample_lecture_sentences[0], n=2))

In [ ]:
from nltk.util import everygrams
padded = list(pad_both_ends(sample_lecture_sentences[0], n=2))
list(everygrams(padded, max_len=n))

In [ ]:
# prepare all of the sentences in the document with flatten
from nltk.lm.preprocessing import flatten
list(flatten(pad_both_ends(sent, n=2) for sent in sample_lecture_sentences))

the padded_everygram_pipeline deals with all of the above steps for us.

https://stackoverflow.com/a/54979617
https://www.nltk.org/api/nltk.lm.html


In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(n, sample_lecture_sentences)

In [ ]:
from nltk.lm import MLE
lm = MLE(n)
lm.fit(train, vocab)
print(lm.vocab)

In [ ]:
lm.vocab.lookup(sample_lecture_sentences[0])

In [ ]:
print(lm.counts)

In [ ]:
lm.counts['a']

In [ ]:
lm.counts[['<s>']]['The']

In [ ]:
lm.score("a")

In [ ]:
lm.score("The", ["<s>"])

In [ ]:
lm.logscore("a")

In [ ]:
lm.logscore("The", ["<s>"])

In [ ]:
# note that this looks like we will need to perform steps manually instead of using padded 
# everygram pipeline so we do not have duplicate padding strings.
lm.generate(15, random_seed=3)

In [ ]:
lm.generate(15, text_seed=['a'], random_seed=3)

In [ ]:
# look at generating text until it is satisfied
start_token = '<s>'
prior_token = start_token
output = []
satisfaction_threshold_log = -12
satisfaction = 0
count = 0
while satisfaction > satisfaction_threshold_log and count < 10:
    output.append(prior_token)
    print("output sequence: %s" % str(output))
    generated = lm.generate(2, text_seed=output, random_seed=3)
    print("generated sequence: %s" % str(generated))
    prior_token = generated[0]
    print("new token: %s" % prior_token)
    print("last token: %s" % output[-1])
    satisfaction = lm.logscore(prior_token, [output[-1]])
    print("satisfaction: %.2f" % satisfaction)
    
print("output: %s" % ' '.join(output))

In [ ]:
# # create a list of all of the sentences with each sentence tokenized.
# def tokenize_transcript(transcript):
#     token_document = [word_tokenize(t) for t in sent_detector.tokenize(transcript)]
#     sents = []
#     start_of_sentence = "<sent>"
#     end_of_sentence = "<\\sent>"
#     for sent in token_document:
#         sent.insert(0, start_of_sentence)
#         sent.append(end_of_sentence)
#         sents.append(sent)
#     return sents
# sample_lecture_sentences = tokenize_transcript(sample_lecture)
# sample_lecture_sentences[1:4]

In [ ]:
# sample_lecture_sentences = [sent for sent in sent_detector.tokenize(sample_lecture)]
# sample_lecture_sentences[1:4]

In [ ]:
# start_of_sentence = "<sent> "
# end_of_sentence = " <\\sent>"
# for i in range(len(sample_lecture_sentences)):
#     sample_lecture_sentences[i] = start_of_sentence + sample_lecture_sentences[i] + end_of_sentence
# sample_lecture_sentences[1:4]

In [ ]:
# start_of_sentence = "<sent> "
# end_of_sentence = " <\\sent>"
# for i in range(len(sample_lecture_sentences)):
#     sample_lecture_sentences[i] = word_tokenize(sample_lecture_sentences[i])
#     sample_lecture_sentences[i].insert(0, start_of_sentence)
#     sample_lecture_sentences[i].append(end_of_sentence)
# sample_lecture_sentences[1:4]

In [ ]:
# n = 4
# from nltk.lm.preprocessing import padded_everygram_pipeline
# train, vocab = padded_everygram_pipeline(n, sample_lecture_sentences)

In [ ]:
# from nltk.lm import MLE
# n = 4
# lm = MLE(n)
# lm.fit(train, vocab)
# print(lm.vocab)

In [ ]:
# lm.vocab.lookup(start_of_sentence)

In [ ]:
# lm.vocab.lookup(sample_lecture_sentences[0])

In [ ]:
# lm.counts[start_of_sentence]

In [ ]:


# sents = []
# for transcript in tqdm(khan['clean_transcript']):
#     sents += tokenize_transcript(transcript)

# print(len(sents))

In [ ]:
# n = 4
# print("Joining sentence data of length %d together" % len(sents))
# tokens = []
# for sent in tqdm(sents):
#     tokens += sent
# print("Computing all N-grams where N is %d or less" % n)
# from nltk.lm.preprocessing import padded_everygram_pipeline
# train, vocab = padded_everygram_pipeline(n, tokens)


# # grams = []
# # for sent in tqdm(sents):
# #     grams += list(everygrams(sent, min_len=2, max_len=n))
# # print("Computing frequency distribution")
# # freq_dist = nltk.FreqDist(grams)
# # freq_dist

In [ ]:
# https://stackoverflow.com/a/54979617
# https://www.nltk.org/api/nltk.lm.html
# from nltk.lm import MLE
# start_token = "<sent>"
# end_token = "<\sent>"
# output_sentence = [start_token]
# n = 4
# lm = MLE(n)
# lm.fit(train, vocab)
# print(lm.vocab)
# possible_next_tokens = {}
# for key in freq_dist.keys():
#     if output_sentence[-1] in key:
#         print(key)
        # possible_next_tokens[key] = freq_dist[key]

# freq_dist